In [18]:
import os
import warnings
import sys
from pathlib import Path
from datetime import datetime, timedelta
import ipywidgets as widgets
import plotly.graph_objs as go
import plotly.express as px
import yfinance as yf
import pandas as pd
import numpy as np
from IPython.display import display
from gamestonk_terminal import api as gst


warnings.filterwarnings("ignore")
module_path = os.path.abspath(os.path.join("../.."))

if module_path not in sys.path:
    sys.path.append(module_path)

In [15]:
def create_layout(fig, width, height, z, ticker):
    z_dict = {
        "IV": "Implied Volatility",
        "OI": "Open Interest",
        "LP": "Last Price",
    }

    fig.update_layout(
        title=f"{z.upper()} Surface for {ticker.upper()}",
        autosize=False,
        width=width,
        height=height,
        scene=dict(
            xaxis_title="Days to Expiration",
            yaxis_title="Strike",
            zaxis_title=z_dict[z],
        ),
    )


def view_chart(z, ticker):
    global last_ticker, df
    if ticker:
        if ticker != last_ticker:
            df = gst.stocks.options.models.yfinance.get_iv_surface(ticker)
            last_ticker = ticker

        if not df.empty:
            z_dict = {
                "IV": df.impliedVolatility,
                "OI": df.openInterest,
                "LP": df.lastPrice,
            }
            fig = go.Figure(
                data=[
                    go.Mesh3d(
                        z=z_dict[z],
                        x=df.dte,
                        y=df.strike,
                        intensity=np.sqrt(z_dict[z]),
                        colorscale=px.colors.sequential.Plasma,
                    )
                ]
            )

            fig.update_traces(showscale=False)
            if os.environ.get("SERVER_SOFTWARE", "jupyter").startswith("voila"):
                create_layout(fig, 1000, 1000, z, ticker)
                fig.show(config={"showTips": False}, renderer="notebook")
            else:
                create_layout(fig, 700, 700, z, ticker)
                fig.show(config={"showTips": False})


last_ticker = ""
opts = ["IV", "OI", "LP"]
z_widget = widgets.Dropdown(options=opts, value="IV", description="Data")

tickers_widget = widgets.Text(description="Ticker", value="TSLA")
controls = widgets.VBox([tickers_widget, z_widget])

stocks_view = widgets.interactive_output(
    view_chart,
    {
        "z": z_widget,
        "ticker": tickers_widget,
    },
)

title_html = "<h1>Option Surface Dashboard</h1>"
app_contents = [
    widgets.HTML(title_html),
    controls,
    stocks_view,
]
app = widgets.VBox(app_contents)
display(app)